Ссылка на данные:

https://drive.google.com/drive/folders/1lcxpKkBuAXYJlagiTc0GkD5EDajS-J_V

In [ ]:
import torch
import torchvision
from torchvision.models.detection import KeypointRCNN
from torchvision.models.detection.anchor_utils import AnchorGenerator

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v3_small(weights=torchvision.models.MobileNet_V3_Small_Weights.DEFAULT).features
# KeypointRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 576

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                    aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
 # use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be ['0']. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)
keypoint_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                         output_size=14,
                                                         sampling_ratio=2)
# put the pieces together inside a KeypointRCNN model
model = KeypointRCNN(backbone,
                     num_classes=2,
                     rpn_anchor_generator=anchor_generator,
                     box_roi_pool=roi_pooler,
                     keypoint_roi_pool=keypoint_roi_pooler, num_keypoints=30)
model.eval()
model.eval()
model.to(DEVICE)
x = [torch.rand(3, 300, 400).to(DEVICE), torch.rand(3, 500, 400).to(DEVICE)]
predictions = model(x)
predictions

In [ ]:
import torch
import torchvision
from torchvision.models.detection import KeypointRCNN
from torchvision.models.detection.anchor_utils import AnchorGenerator

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
keypointrcnn_resnet50_fpn(pretrained=False, num_keypoints=30)
model.eval()
model.eval()
model.to(DEVICE)
x = [torch.rand(3, 300, 400).to(DEVICE), torch.rand(3, 500, 400).to(DEVICE)]
predictions = model(x)
predictions

In [ ]:
from google.colab import drive



drive.mount('/content/drive')
id="/content/drive/MyDrive/"

Mounted at /content/drive


In [ ]:
from torch.utils.data import Dataset, DataLoader
import os
import random
import cv2
import json
import pandas as pd
from torchvision import transforms
class ClassDataset(Dataset):
    def __init__(self, annotation, image_dir):                
        self.images=[]
        self.count=0
        self.targets = []
        self.imgs_files=[]
        annotation=pd.read_csv(annotation)
        annotation=annotation.to_numpy()
        for i in range(len(annotation)):
          target={}
          if "cow" in annotation[i][0]:
            img=cv2.imread(image_dir+"/"+annotation[i][0])
            self.imgs_files.append(image_dir+"/"+annotation[i][0])
          else:
            img=cv2.imread(image_dir+"/"+annotation[i][0].capitalize())
            self.imgs_files.append(image_dir+"/"+annotation[i][0].capitalize())

          print(image_dir+"/"+annotation[i][0].capitalize())
          h, w = img.shape[:2]
          size=224
          img = cv2.resize(img, (size, size))
          F = transforms.ToTensor()
          keypoints=[]
          x_min=1.0
          y_min=1.0
          x_max=0.0
          y_max=0.0
          for j in range(1, len(annotation[i])):
            if j%3==2:
              k=[]
              k.append(annotation[i][j]/w)
              x_min=min(x_min, annotation[i][j]/w)
              x_max=max(x_max, annotation[i][j]/w)
            if j%3==0:
              k.append(annotation[i][j]/h)
              y_min=min(y_min,annotation[i][j]/h)
              y_max=max(y_max,annotation[i][j]/h)

              k.append(annotation[i][j-2])
              keypoints.append(k)
          bboxes=torch.Tensor([[x_min,y_min,x_max,y_max]])
          target["boxes"] = bboxes
          target["labels"] = torch.as_tensor([1 for _ in bboxes], dtype=torch.int64)
          target["image_id"] = torch.tensor([i])
          target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
          target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
          target["keypoints"] = torch.as_tensor([keypoints], dtype=torch.float32) 
          keys=target.keys()
          for key in keys:
            target[key]     
          img = F(img)
          img
          self.images.append(img)
          self.targets.append(target)
          self.count+=1
          if self.count==200:
            break
        #F = transforms.ToTensor()
         
        # self.target["boxes"] = bboxes
        # self.target["labels"] = torch.as_tensor([1 for _ in bboxes], dtype=torch.int64) # all objects are glue tubes
        # self.target["image_id"] = torch.tensor([idx])
        # self.target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        # self.target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        # self.target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)        
        # self.img = F.ToTensor(img)
        
  
    
    def __getitem__(self, idx):
            return self.images[idx], self.targets[idx]
    
    def __len__(self):
        return self.count
    def get_image_files(self):
      return self.imgs_files

In [ ]:
from zmq.constants import NULL
annotation=pd.read_csv(id+"training/data_set9/test_data1.csv")

print(annotation)
annotation=annotation.to_numpy()
data=[]
for i in range(len(annotation)):
  #line=line.remove('"')
  line=annotation[i][0].split(";")

  data.append(line)
data=pd.DataFrame(data)
for i in range(1, 91):
  if i%3==1:
    data[i] = data[i].astype (int)
  else:
    data[i] = data[i].astype (float)
data=data.drop(columns=[91])
data.to_csv("test_dataset.csv")
print(data.head())

    german+shepherd_10032.jpg;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;742.2907488986784;260.4669603524229;1;194.27312775330395;240.2026431718062;0;0;0;0;0;0;0;0;0;
0    lion_10504.jpg;1;667.018018018018;538.35135135...                                                                                                                                                                                                                             
1    collie_10630.jpg;1;617.0616740088105;310.63876...                                                                                                                                                                                                                             
2    cow_10271.jpg;1;432.44827586206895;794.8275862...                                                                                                                      

In [ ]:
dataset_train = ClassDataset(id+"training/dataset/train_dataset.csv", id+"training/animals")
# dataset_test = ClassDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)
data_loader_train = DataLoader(dataset_train, batch_size=8, shuffle=True)
# data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [ ]:
data_loader_train = DataLoader(dataset_train, batch_size=16, shuffle=False)


In [ ]:
dataset_test = ClassDataset(id+"training/dataset/test_dataset.csv", id+"training/animals")


/content/drive/MyDrive/training/animals/Lion_10504.jpg
/content/drive/MyDrive/training/animals/Collie_10630.jpg
/content/drive/MyDrive/training/animals/Cow_10271.jpg
/content/drive/MyDrive/training/animals/Chihuahua_10272.jpg
/content/drive/MyDrive/training/animals/Leopard_10089.jpg
/content/drive/MyDrive/training/animals/Lion_10079.jpg
/content/drive/MyDrive/training/animals/Polar+bear_10133.jpg
/content/drive/MyDrive/training/animals/Ox_10318.jpg
/content/drive/MyDrive/training/animals/Deer_10553.jpg
/content/drive/MyDrive/training/animals/Deer_10102.jpg
/content/drive/MyDrive/training/animals/Siamese+cat_10044.jpg
/content/drive/MyDrive/training/animals/Ox_10226.jpg
/content/drive/MyDrive/training/animals/Moose_10309.jpg
/content/drive/MyDrive/training/animals/Sheep_10221.jpg
/content/drive/MyDrive/training/animals/Lion_10113.jpg
/content/drive/MyDrive/training/animals/Bobcat_10248.jpg
/content/drive/MyDrive/training/animals/Zebra_10343.jpg
/content/drive/MyDrive/training/animals/Mo

In [ ]:
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False)

In [ ]:
#https://colab.research.google.com/drive/1YtJrw6oQQbTfTm3uwir7yXMHrLO0NmFp?hl=ru#scrollTo=IFhO39WYbZ9A
#https://medium.com/@alexppppp/how-to-train-a-custom-keypoint-detection-model-with-pytorch-d9af90e111da

In [ ]:


import datetime
import errno
import os
import time
from collections import defaultdict, deque

import torch
import torch.distributed as dist


class SmoothedValue:
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device="cuda")
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median, avg=self.avg, global_avg=self.global_avg, max=self.max, value=self.value
        )
def collate_fn(batch):
    return tuple(zip(*batch))

def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]
    data_list = [None] * world_size
    dist.all_gather_object(data_list, data)
    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.inference_mode():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger:
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{attr}'")

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(f"{name}: {str(meter)}")
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ""
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt="{avg:.4f}")
        data_time = SmoothedValue(fmt="{avg:.4f}")
        space_fmt = ":" + str(len(str(len(iterable)))) + "d"
        if torch.cuda.is_available():
            log_msg = self.delimiter.join(
                [
                    header,
                    "[{0" + space_fmt + "}/{1}]",
                    "eta: {eta}",
                    "{meters}",
                    "time: {time}",
                    "data: {data}",
                    "max mem: {memory:.0f}",
                ]
            )
        else:
            log_msg = self.delimiter.join(
                [header, "[{0" + space_fmt + "}/{1}]", "eta: {eta}", "{meters}", "time: {time}", "data: {data}"]
            )
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                if torch.cuda.is_available():
                    print(
                        log_msg.format(
                            i,
                            len(iterable),
                            eta=eta_string,
                            meters=str(self),
                            time=str(iter_time),
                            data=str(data_time),
                            memory=torch.cuda.max_memory_allocated() / MB,
                        )
                    )
                else:
                    print(
                        log_msg.format(
                            i, len(iterable), eta=eta_string, meters=str(self), time=str(iter_time), data=str(data_time)
                        )
                    )
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print(f"{header} Total time: {total_time_str} ({total_time / len(iterable):.4f} s / it)")


def collate_fn(batch):
    return tuple(zip(*batch))


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__

    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop("force", False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if "RANK" in os.environ and "WORLD_SIZE" in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ["WORLD_SIZE"])
        args.gpu = int(os.environ["LOCAL_RANK"])
    elif "SLURM_PROCID" in os.environ:
        args.rank = int(os.environ["SLURM_PROCID"])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print("Not using distributed mode")
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = "nccl"
    print(f"| distributed init (rank {args.rank}): {args.dist_url}", flush=True)
    torch.distributed.init_process_group(
        backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size, rank=args.rank
    )
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

In [ ]:
import datetime
import errno
import os
import time
from collections import defaultdict, deque

import torch
import torch.distributed as dist


class SmoothedValue:
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device="cuda")
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median, avg=self.avg, global_avg=self.global_avg, max=self.max, value=self.value
        )


def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]
    data_list = [None] * world_size
    dist.all_gather_object(data_list, data)
    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.inference_mode():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger:
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{attr}'")

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(f"{name}: {str(meter)}")
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ""
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt="{avg:.4f}")
        data_time = SmoothedValue(fmt="{avg:.4f}")
        space_fmt = ":" + str(len(str(len(iterable)))) + "d"
        if torch.cuda.is_available():
            log_msg = self.delimiter.join(
                [
                    header,
                    "[{0" + space_fmt + "}/{1}]",
                    "eta: {eta}",
                    "{meters}",
                    "time: {time}",
                    "data: {data}",
                    "max mem: {memory:.0f}",
                ]
            )
        else:
            log_msg = self.delimiter.join(
                [header, "[{0" + space_fmt + "}/{1}]", "eta: {eta}", "{meters}", "time: {time}", "data: {data}"]
            )
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                if torch.cuda.is_available():
                    print(
                        log_msg.format(
                            i,
                            len(iterable),
                            eta=eta_string,
                            meters=str(self),
                            time=str(iter_time),
                            data=str(data_time),
                            memory=torch.cuda.max_memory_allocated() / MB,
                        )
                    )
                else:
                    print(
                        log_msg.format(
                            i, len(iterable), eta=eta_string, meters=str(self), time=str(iter_time), data=str(data_time)
                        )
                    )
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print(f"{header} Total time: {total_time_str} ({total_time / len(iterable):.4f} s / it)")


def collate_fn(batch):
    return tuple(zip(*batch))


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__

    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop("force", False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if "RANK" in os.environ and "WORLD_SIZE" in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ["WORLD_SIZE"])
        args.gpu = int(os.environ["LOCAL_RANK"])
    elif "SLURM_PROCID" in os.environ:
        args.rank = int(os.environ["SLURM_PROCID"])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print("Not using distributed mode")
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = "nccl"
    print(f"| distributed init (rank {args.rank}): {args.dist_url}", flush=True)
    torch.distributed.init_process_group(
        backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size, rank=args.rank
    )
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

In [ ]:
import math, sys


def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq,  scaler=None):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )
    
    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        items=targets.keys()
        new_targets=[]
        
        target={}
        for item in items:
          for i in range(len(images)):
            if item=="boxes":
              new_targets.append(target)
            new_targets[i][item]=targets[item][i]
          
        targets=new_targets
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger



In [ ]:
num_epochs=20
params = [p.to(DEVICE) for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader_train, DEVICE, epoch, print_freq=1000)
    lr_scheduler.step()


OutOfMemoryError: ignored

In [ ]:
model.eval()#evaluate(model, data_loader_test, DEVICE)
with torch.no_grad():
  F = transforms.ToTensor()
  from PIL import Image
  import torchvision.transforms as T
  transform = T.ToPILImage()
  image=dataset_train.get_image_files()[10]
  image=cv2.imread((image))
  image=cv2.resize(image, (224, 224))
  trans1 = transforms.ToTensor()
  print(trans1(image))
  print(target)
  print(model([trans1(image).to(DEVICE)]))

tensor([[[0.9333, 0.9373, 0.9412,  ..., 0.9294, 0.9294, 0.9255],
         [0.9373, 0.9255, 0.9412,  ..., 0.9255, 0.9333, 0.9294],
         [0.9490, 0.9373, 0.9294,  ..., 0.9294, 0.9294, 0.9294],
         ...,
         [0.5137, 0.2392, 0.2549,  ..., 0.3098, 0.2039, 0.3176],
         [0.4275, 0.4941, 0.2353,  ..., 0.2824, 0.1765, 0.1882],
         [0.4627, 0.2745, 0.2196,  ..., 0.3255, 0.1647, 0.1882]],

        [[0.9333, 0.9294, 0.9333,  ..., 0.9137, 0.9098, 0.9098],
         [0.9333, 0.9294, 0.9333,  ..., 0.9098, 0.9059, 0.9059],
         [0.9373, 0.9333, 0.9294,  ..., 0.9059, 0.9098, 0.9020],
         ...,
         [0.7843, 0.5255, 0.5059,  ..., 0.5294, 0.4314, 0.5176],
         [0.6667, 0.7216, 0.5647,  ..., 0.4706, 0.4314, 0.4471],
         [0.6667, 0.6510, 0.5137,  ..., 0.5490, 0.5647, 0.4431]],

        [[0.9333, 0.9294, 0.9333,  ..., 0.8941, 0.8902, 0.8902],
         [0.9294, 0.9294, 0.9333,  ..., 0.8902, 0.8863, 0.8863],
         [0.9216, 0.9294, 0.9294,  ..., 0.8863, 0.8863, 0.

In [ ]:
model.eval()
with torch.no_grad():
  image, target=dataset_train.__getitem__(4)
  print(model([image.to(DEVICE)]))

[{'boxes': tensor([[4.4424e-03, 1.0510e-02, 2.3376e+00, 1.5848e+00],
        [8.9172e-03, 3.1159e-02, 4.6922e+00, 4.6987e+00],
        [1.4326e-02, 5.5191e-02, 7.5386e+00, 8.3226e+00],
        [1.8049e-02, 3.4192e-02, 1.4075e+01, 6.1018e+00],
        [0.0000e+00, 1.5283e+00, 1.1542e+01, 1.5469e+01],
        [0.0000e+00, 5.4665e-02, 2.2775e+01, 1.0845e+01],
        [3.3766e+00, 4.8904e-02, 1.5151e+01, 1.4088e+01],
        [0.0000e+00, 4.0022e+00, 4.6131e+00, 1.2583e+01],
        [4.0461e+00, 2.4690e-02, 1.2914e+01, 4.1225e+00],
        [7.5304e+00, 8.3052e-03, 1.1823e+01, 1.1064e+00],
        [8.6557e+00, 1.8071e-02, 1.0646e+01, 2.6565e+00],
        [0.0000e+00, 7.4670e+00, 9.1397e-01, 1.1079e+01],
        [5.0277e+00, 3.8577e+00, 1.4059e+01, 1.2589e+01],
        [0.0000e+00, 9.7097e-02, 6.1209e+00, 2.1062e+01],
        [0.0000e+00, 8.7001e+00, 1.7206e+00, 1.0309e+01]], device='cuda:0'), 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), 'scores': tensor([

In [ ]:
x = [torch.rand(3, 300, 400).to(DEVICE), torch.rand(3, 500, 400).to(DEVICE)]
predictions = model(x)
print(predictions)

OutOfMemoryError: ignored

In [ ]:
evaluate(model, data_loader_test, device=DEVICE)

NameError: ignored